In [2]:
import pandas as pd
import pandas.io.sql
import pandas.io.formats.style
import pymssql as sapsql 
import numpy as np
from IPython.display import display
from jinja2 import Environment, FileSystemLoader
import pdfkit
import telegram 


env = Environment(loader=FileSystemLoader('/data/jupyter/'))
#template = env.get_template("template.html")

datacompany = ({#'host':'192.168.250.40' , 'dbname':'IGU_LIVE_20221204','user':'sa','password':'B1admin'},
              'host':'192.168.250.40' , 'dbname':'IGU_NEW','user':'sa','password':'B1admin'},
    )

datalist=[]
 
for company in datacompany:
    msg_sql= """

select
    a.docnum,
    a.docdate,
    a.cardcode,
    a.doctotal,
    month (a.docdate) mth,
    c.groupname,
    case 
  when c.groupname IN ('Restaurant', 'Hotel', 'Catering') then 'HOREKA'
  when c.groupname IN ('Distributor', 'Manufacture', 'Umum') then 'WET'
  when c.groupname = 'QSR' then 'QSR'
  when c.groupname = 'Supermarket' then 'SUPERMARKET'
  when c.groupname = 'Group Jkt' then 'MEATSHOP'
  when c.groupname = 'Cabang LK' then 'CABANG'
  when c.groupname = 'Pastry & Bakery' then 'PASTRY'
  when c.groupname = 'Ecommerce' then 'ONLINE'
  else 'OTHERS' 
  end status
                     

    from ordr a  
     inner join  ocrd b on a.cardcode = b.cardcode
     inner join ocrg c on b.groupcode = c.groupcode
                
WHERE convert(varchar,a.DocDate,112) between '20230101' and '20230131' and canceled = 'N'

"""

 
    

    conn = sapsql.connect(host=company["host"] , user=company["user"] , password=company["password"], database=company["dbname"] )
    cursor = conn.cursor()     
    data = pandas.io.sql.read_sql(msg_sql,conn)
    datalist.append(data)

df2m = pd.concat(datalist)  

options = {
            'page-size': 'legal',
            'orientation': 'landscape',
            }
#html_out = template.render(template_vars)
textstring = ""
#test = df.pivot_table(index=["groupname",'cardname'],columns=["PymntGroup"],aggfunc=np.sum,  values=["Balance"],fill_value="0",margins=True)

df2m

,docnum,docdate,cardcode,doctotal,mth,groupname,status
0,301100001,2023-01-01,DS0067,500000000.0,1,Distributor,WET
1,301100002,2023-01-03,EC0014,11875000.0,1,Ecommerce,ONLINE
2,301100003,2023-01-01,RS1056,1575000.0,1,Restaurant,HOREKA
3,301100004,2023-01-01,OT0159,1920000.0,1,Others,OTHERS
4,301100005,2023-01-01,RS0913,27930000.0,1,Restaurant,HOREKA
...,...,...,...,...,...,...,...
24038,301124673,2023-01-25,SM0095,3048282.0,1,Supermarket,SUPERMARKET
24039,301124674,2023-01-25,RS2948,1110000.0,1,Restaurant,HOREKA
24040,301124675,2023-01-25,RS2948,700000.0,1,Restaurant,HOREKA
24041,301124676,2023-01-27,RS2259,905760.0,1,Restaurant,HOREKA


In [7]:
df2m.to_excel (r'02122022.xlsx', index=True, header=True, merge_cells=True)

In [3]:
df2m = df2m.fillna(0)

Grup = df2m.groupby("mth")

df2 = Grup.get_group(1)

df2 ['date'] = df2['docdate'].dt.strftime("%d-%b")

df2

pivot1 = df2.pivot_table(index=['date'], columns= ['status'], values=['doctotal'], aggfunc=['sum'], margins = True)
pivot1 = pivot1.droplevel(0,axis=1)
pivot2 = pivot1.droplevel(0,axis=1)

pivot2 ['% cbg'] = (pivot2['CABANG']/pivot2['All'])*100
pivot2 ['% hrk'] = (pivot2['HOREKA']/pivot2['All'])*100
pivot2 ['% mts'] = (pivot2['MEATSHOP']/pivot2['All'])*100
pivot2 ['% onl'] = (pivot2['ONLINE']/pivot2['All'])*100
pivot2 ['% OTH'] = (pivot2['OTHERS']/pivot2['All'])*100
pivot2 ['% PST'] = (pivot2['PASTRY']/pivot2['All'])*100
pivot2 ['% QSR'] = (pivot2['QSR']/pivot2['All'])*100
pivot2 ['% SPM'] = (pivot2['SUPERMARKET']/pivot2['All'])*100
pivot2 ['% WET'] = (pivot2['WET']/pivot2['All'])*100
pivot2 ['% ALL'] = (pivot2['All']/pivot2['All'])*100

pivot2 = pivot2[['All','% ALL','CABANG','% cbg','HOREKA','% hrk','MEATSHOP','% mts','ONLINE','% onl','OTHERS','% OTH','PASTRY','% PST','QSR','% QSR','SUPERMARKET','% SPM', 'WET', '% WET']]

pivot2

#pivot2.info()

#out = (
#    pivot2.pipe(lambda d: d.div(d['All'], axis='index'))
#    .applymap('{:.0%}'.format)
#)

#out.info()

#tes = pd.concat (['out','pivot2'])
#tes

gb12 = df2m.groupby (['mth'])['docnum'].count()

gb22 = df2m.groupby (['mth'])['doctotal'].sum()

GB1 = pd.concat([gb22, gb12], axis=1)

GB1['%'] = (GB1['doctotal']/GB1['doctotal'].sum())*100

GB1 = GB1[['doctotal','%','docnum']]

GB1 = GB1.append([GB1.sum().rename('Total'),GB1.mean().rename('Avg')])

GB1

gb1 = df2.groupby (['date'])['docnum'].count()

gb2 = df2.groupby (['date'])['doctotal'].sum()

GB = pd.concat([gb2, gb1], axis=1)

GB['%'] = (GB['doctotal']/GB['doctotal'].sum())*100

GB = GB[['doctotal','%','docnum']]

GB = GB.append(GB.sum().rename('Total'))

slc1 = ['HOREKA','WET','QSR','SUPERMARKET','MEATSHOP','CABANG','PASTRY','ONLINE','OTHERS',]
#slc2 = ['% hrk', '% WET','% QSR', '% SPM','% mts', '% cbg','% PST','% onl','% OTH']
#slc = ['Distributor', 'QSR','Catering','Cabang LK','Coffee Shop','Ecommerce','FC','Group Jkt','Hotel','Karyawan','Manufacture','Others','Pastry & Bakery','QSR','Restaurant','Supermarket','TokoTani']
sf1 = pivot2.style.highlight_max(color='pink',axis=1,subset=slc1).format("{:,.0f}") 
#sf2 = pivot2.style.highlight_max(color='pink',axis=1,subset=slc2).format("{:,.0f}") 

sf1

status,All,% ALL,CABANG,% cbg,HOREKA,% hrk,MEATSHOP,% mts,ONLINE,% onl,OTHERS,% OTH,PASTRY,% PST,QSR,% QSR,SUPERMARKET,% SPM,WET,% WET
date,,,,,,,,,,,,,,,,,,,,
01-Jan,"4,416,413,517",100,"613,602,406",14,"3,060,209,890",69,nan,nan,nan,nan,"5,760,000",0,"3,123,200",0,"13,701,403",0,"220,016,619",5,"500,000,000",11
02-Jan,"72,910,021",100,nan,nan,"68,494,021",94,nan,nan,nan,nan,"4,416,000",6,nan,nan,nan,nan,nan,nan,nan,nan
03-Jan,"13,040,838,546",100,"4,321,680,670",33,"2,817,822,159",22,"1,009,181,980",8,"47,153,562",0,"319,426,278",2,"96,531,132",1,"1,132,052,568",9,"1,033,546,145",8,"2,263,444,053",17
04-Jan,"8,895,581,934",100,"2,029,603,214",23,"1,614,865,687",18,"389,941,180",4,"144,530,592",2,"363,741,403",4,"51,068,887",1,"665,941,506",7,"653,539,727",7,"2,982,349,737",34
05-Jan,"10,517,652,671",100,"5,517,716,336",52,"2,260,900,227",21,"440,507,852",4,"69,466,510",1,"319,740,188",3,"54,462,083",1,"268,021,420",3,"518,520,429",5,"1,068,317,626",10
06-Jan,"10,312,716,488",100,"744,400,720",7,"2,316,241,526",22,"274,038,119",3,"162,084,599",2,"269,042,739",3,"102,393,555",1,"614,766,527",6,"352,391,962",3,"5,477,356,740",53
07-Jan,"7,900,131,589",100,"2,029,361,927",26,"3,112,059,761",39,"619,700,000",8,"46,550,018",1,"292,648,623",4,"1,558,000",0,"121,476,504",2,"100,941,261",1,"1,575,835,495",20
08-Jan,"143,046,129",100,nan,nan,"19,501,847",14,nan,nan,nan,nan,"91,544,282",64,nan,nan,nan,nan,nan,nan,"32,000,000",22
09-Jan,"4,744,353,175",100,"784,514,834",17,"2,046,924,570",43,"50,569,200",1,"30,410,901",1,"219,978,417",5,"40,739,980",1,"454,893,511",10,"167,833,147",4,"948,488,616",20


In [4]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('SO Jan24.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
sf1.to_excel(writer, sheet_name = 'groupby', merge_cells=True)
GB.to_excel(writer, sheet_name='Monthly',merge_cells=True)
GB1.to_excel(writer, sheet_name='Monthly',merge_cells=True, startrow=1+len(GB)+3, startcol=0)

# Get the xlsxwriter objects from the dataframe writer object.
workbook  = writer.book
worksheet = writer.sheets['groupby']

# Add some cell formats.
format1 = workbook.add_format({'num_format': '#,##0.00'})

# Set the column width and format.
worksheet.set_column('B:R', 18, format1)

writer.save()

#pivot2 = df2.pivot_table(index=['date'], columns= ['doctotal'], values=['doctotal'], aggfunc='sum',margins=True)

#pivot2
#pdfkit.from_string(html_out,path_PDFfile1,options=options,css=cssfile) 
#df.to_excel (r'item order 2020_1.xlsx', index=True, header=True, merge_cells=False)

In [81]:
df = pd.DataFrame(np.random.randint(100, size=(6,8)))

sf = df.style.highlight_min(color='red',axis=1)\
.highlight_max(color='green', axis=1)

sf

,0,1,2,3,4,5,6,7
0,77,74,87,64,30,44,96,92
1,59,89,29,40,56,73,2,16
2,3,24,72,12,76,76,71,53
3,73,65,22,17,20,38,69,47
4,75,42,27,49,33,67,81,75
5,92,82,48,49,93,97,55,42


In [13]:
sf1.to_excel (r'KStock 2022-Jan-Jun.xlsx', index=True, header=True, merge_cells=True, float_format = "{:,.0f}")

TypeError: not all arguments converted during string formatting

In [34]:
import pandas as pd
import numpy as np
import matplotlib as mpl

df = pd.DataFrame([[38.0, 2.0, 18.0, 22.0, 21, np.nan],[19, 439, 6, 452, 226,232]],
                  index=pd.Index(['Tumour (Positive)', 'Non-Tumour (Negative)'], name='Actual Label:'),
                  columns=pd.MultiIndex.from_product([['Decision Tree', 'Regression', 'Random'],['Tumour', 'Non-Tumour']], names=['Model:', 'Predicted:']))
df.style

df

Model:                Decision Tree            Regression            Random  \
Predicted:                   Tumour Non-Tumour     Tumour Non-Tumour Tumour   
Actual Label:                                                                 
Tumour (Positive)              38.0        2.0       18.0       22.0     21   
Non-Tumour (Negative)          19.0      439.0        6.0      452.0    226   

Model:                            
Predicted:            Non-Tumour  
Actual Label:                     
Tumour (Positive)            NaN  
Non-Tumour (Negative)      232.0

In [35]:
worksheet.conditional_format('A1:F10', {'type':   'top',
                                        'value':  10,
                                        'format': format1})

In [36]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('SO Juli21.xlsx', engine='xlsxwriter')
sf1.to_excel(writer, sheet_name = 'groupby', merge_cells=True)

# Get the xlsxwriter objects from the dataframe writer object.
workbook  = writer.book
worksheet = writer.sheets['groupby']

# Add some cell formats.
format1 = workbook.add_format({'num_format': '#,##0.00'})

# Set the column width and format.
worksheet.set_column('B:Q', 18, format1)

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [28]:
data =  pd.DataFrame(np.random.randn(5, 4), columns=list('KLMX'))
#print (data)

def highlight_cols(s):
    color = 'red'
    return 'background-color: %s' % color

df = data.style.applymap(highlight_cols, subset=pd.IndexSlice[:, ['K','M']])

df

,K,L,M,X
0,0.669568,0.459564,2.344865,0.890106
1,-1.240353,-0.954571,0.432562,0.201852
2,-0.777685,0.851461,-1.813619,0.566242
3,2.185036,0.318553,-1.226946,-1.615131
4,-0.583693,-0.141713,-1.192871,1.067421


In [46]:
import pandas as pd
import numpy as np
import seaborn as sns

df = pd.DataFrame({"A": ["foo", "foo", "foo", "foo", "foo",
                      "bar", "bar", "bar", "bar"],
                "B": ["one", "one", "one", "two", "two",
                      "one", "one", "two", "two"],
                "C": ["small", "large", "large", "small",
                      "small", "large", "small", "small",
                     "large"],
                "D": [1, 2, 2, 3, 3, 4, 5, 6, 7]})

df = df.pivot_table(values='D', index=['A','B'], columns=['C'], aggfunc=np.sum, margins=True, fill_value = 0)


#df = df.style.background_gradient(cmap='Blues', axis=0, subset=pd.IndexSlice[u[:-1], :])

df

C        large  small  All
A   B                     
bar one      4      5    9
    two      7      6   13
foo one      4      1    5
    two      0      6    6
All         15     18   33

In [77]:
u = df.index.get_level_values(0)

u=(df.style.background_gradient(
  subset = pd.IndexSlice[u[:-1], 'large',],
  cmap = sns.light_palette('red', as_cmap = True)))


u

KeyError: "None of [Index(['large'], dtype='object')] are in the [columns]"

In [92]:
df3 = pd.DataFrame(np.random.randn(4,4),
                   pd.MultiIndex.from_product([['A', 'B'], ['r1', 'r2']]),
                   columns=['c1','c2','c3','c4'])

slice_ = ['c3', 'c4','c1']
df8=df3.style.highlight_max(color='red', axis=1, subset=slice_)
         

df8

In [2]:
import pandas as pd
import pandas.io.sql
import pandas.io.formats.style
import pymssql as sapsql 
import numpy as np
from IPython.display import display
from jinja2 import Environment, FileSystemLoader
import pdfkit
import telegram 


env = Environment(loader=FileSystemLoader('/data/jupyter/'))
#template = env.get_template("template.html")

datacompany = (
                {'host':'192.168.250.10' , 'dbname':'IGU_LIVE','user':'sa','password':'B1admin'},
                               )

datalist=[]
 
for company in datacompany:
    msg_sql= """

declare @dateto varchar(10) 

set @dateto = '20211231'


select a.docnum , 
       convert(varchar,a.docdate,23) docdate ,
       month(a.docdate) imonth,
       year(a.docdate) iyear,
       a.numatcard ,
       a.doctotal - a.vatsum dpp , 
       a.vatsum  ppn ,
       a.doctotal ,
       a.doctotal - isnull (b.reconsum,0) balance
from oinv a 
left outer join 
( select SrcObjAbs , sum(a.reconSum) reconSum from ITR1 a 
    inner join oitr b on a.reconnum = b.reconnum 
    where b.canceled ='N' and year(b.ReconDate)<=2021 and a.SrcObjTyp=13
    group by SrcObjAbs 
)b on a.docentry = b.SrcObjAbs
where  a.doctotal - isnull (b.reconsum,0)<>0
and convert(varchar,a.docdate,112)<=@dateto
union all

select a.docnum , 
       convert(varchar,a.docdate,23) docdate ,
        month(a.docdate) imonth,
        year(a.docdate) iyear,
        a.numatcard ,
       a.doctotal - a.vatsum dpp , 
       a.vatsum  ppn ,
       -1 * a.doctotal ,
        -1 * (a.doctotal - isnull (b.reconsum,0)) balance
from orin a 
left outer join 
( select SrcObjAbs , sum(a.reconSum) reconSum from ITR1 a 
    inner join oitr b on a.reconnum = b.reconnum 
    where b.canceled ='N' and year(b.ReconDate)<=2021 and a.SrcObjTyp=14
    group by SrcObjAbs 
)b on a.docentry = b.SrcObjAbs
where  a.doctotal - isnull (b.reconsum,0)<>0
and convert(varchar,a.docdate,112)<=@dateto

"""

 
    

    conn = sapsql.connect(host=company["host"] , user=company["user"] , password=company["password"], database=company["dbname"] )
    cursor = conn.cursor()     
    data = pandas.io.sql.read_sql(msg_sql,conn)
    datalist.append(data)

df2m = pd.concat(datalist)  

options = {
            'page-size': 'legal',
            'orientation': 'landscape',
            }
#html_out = template.render(template_vars)
textstring = ""
#test = df.pivot_table(index=["groupname",'cardname'],columns=["PymntGroup"],aggfunc=np.sum,  values=["Balance"],fill_value="0",margins=True)

df2m

,docnum,docdate,imonth,iyear,numatcard,dpp,ppn,doctotal,balance
0,1901101083,2019-01-15,1,2019,SO201901011014,6469200.0,0.0,6469200.0,6469200.0
1,1812505206,2018-12-28,12,2018,SO201812024338,1044053.0,0.0,1044053.0,1044053.0
2,1801203014,2018-01-19,1,2018,SO201801011004,686000.0,0.0,686000.0,686000.0
3,1803504827,2018-03-24,3,2018,SO201803020083,336600.0,0.0,336600.0,336600.0
4,1803104352,2018-03-27,3,2018,SO201803021738,512000.0,51200.0,563200.0,563200.0
...,...,...,...,...,...,...,...,...,...
47690,1909100591,2019-09-20,9,2019,None,24500.0,2450.0,-26950.0,-26950.0
47691,2112100774,2021-12-15,12,2021,None,351500.0,35150.0,-386650.0,-386650.0
47692,2111100736,2021-11-08,11,2021,None,24675.0,2467.5,-27142.5,-27142.5
47693,2111100779,2021-11-08,11,2021,None,34500.0,3450.0,-37950.0,-37950.0


In [4]:
df2m.to_excel (r'Aging.xlsx', index=True, header=True, merge_cells=True)